<a href="https://colab.research.google.com/github/irdinainsharuki/AdvancedMLBCNittanyAi/blob/main/pneumonia_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🔍 Objective:
This project will **introduce you to CNNs**.

### 📌 What You’ll Do:
1. Define suitable transforms/augmentations for your `train` and `test` images.
2. Pass these images into PyTorch `DataLoaders` for batch processing.
3. Implement `CNN` class architecture for pneumonia image classification.
4. Train and validate your model.

💡 **PLEASE PLEASE PLEASE look things up!!! This is YOUR learning experience.**

---

In [18]:
import torch
import kagglehub
import torch.nn as nn

path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

train_path = f'{path}/chest_xray/train/'
test_path = f'{path}/chest_xray/test/'


#### 📌 ***TASK 1 - DATA PREPROCESSING***

Define image augmentations in the cell below using two variables:  

- **`transform_train`**: Stores transforms for training images. You can include any augmentations you prefer.  
- **`transform_test`**: Stores transforms for your test images. As a best practice, limit these transformations to only the essential ones from `transform_train`.

Lastly, be sure to convert all images to [tensors](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) via the `transforms.ToTensor()` transform. Don't know transforms? [Click here](https://pytorch.org/vision/stable/transforms.html).

In [19]:
from torchvision import transforms
# Define transforms for training and testing datasets
transform_train = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])
...

Ellipsis

Now, we'll pass our images and transforms into a `DataLoader`, which allows us to train our model in batches.
Most of the code is done for you, but [click here](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) to learn more.

In [20]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

num_batches = 16  # Feel free to change this

train_dataset = ImageFolder(root=train_path, transform=transform_train) # Your Transformed train images
train_loader = DataLoader(dataset=train_dataset, batch_size=num_batches, shuffle=True)

test_dataset = ImageFolder(root=test_path, transform=transform_test) # Your transformed test images
test_loader = DataLoader(dataset=test_dataset, batch_size=num_batches, shuffle=True)

#### 📌 ***TASK 2 - CNN Architecture***

This is where you have all the creative freedom in the world. Here are some good questions to ask yourself:

- How many [channels](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-49AG77e4Qp2e7EkARdFsTA) should go into the input layer?
- What measures can I take to avoid [overfitting](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-YdAbhqQzRZaEq39BEQzA6w)?
- What matters to me? (Training Speed / Performance tradeoffs)
- **CONVOLUTION. ACTIVATION FUNCTION. POOLING!!!** 📢📢📢

Not comfortable with PyTorch? [Here](https://youtu.be/mozBidd58VQ?si=TE2_81TEQko1eDXT). Go and make me the best [CNN](https://www.datacamp.com/tutorial/introduction-to-convolutional-neural-networks-cnns) I've ever seen :)

In [21]:
import torch
import torch.nn as nn

# Remember what Pooling does to feature maps!!!
image_width = 150
image_height = 150

class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully connected layers
        # After three pooling layers, the image dims are reduced by a factor of 8 (150 // 8 = 18)
        self.fc1 = nn.Linear(128 * (image_width // 8) * (image_height // 8), 256)
        self.fc2 = nn.Linear(256, 1)

        # Dropout to avoid overfitting
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.flatten(x, start_dim=1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

# Instantiate the model
model = PneumoniaCNN()
print(model)
...

PneumoniaCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=41472, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


Ellipsis

#### 📌 ***TASK 3 - DEFINE TRAIN FUNCTION***  

Define `process_forward_phase` and `train` to update model weights with each new [epoch/iteration](https://www.perplexity.ai/search/i-m-a-curious-naiss-mlb-studen-7SJECNYrS1iYxUR032dp7A). Here are the steps:

- **Forward pass:** Here, our batch is taken through the network to output a prediction (Normal/Pneumonia)
- **Backward pass:** The model goes "What's our loss? Hmmm... Not quite what I want. This means my `weights` aren't adjusted properly. Let me propagate my `loss` backward in hopes of correcting my weights."

We use **`f1_score`** as the primary metric and also display **`accuracy`** for comparison. Most steps are outlined for you—just follow the structure provided!

In [22]:
from tqdm import tqdm # Visualize training progress
from sklearn.metrics import f1_score, accuracy_score

def process_forward_pass(model, batch, criterion):
    """
    This is a helper function to abstract the "forward"
    phase of the training loop. This function also returns
    the loss, predictions, and labels seen in the batch.
    """

    images, labels = batch
    labels = labels.float().unsqueeze(1)  # Convert labels to float and add a dimension for compatibility
    outputs = model(images)
    loss = criterion(outputs, labels)
    preds = (outputs > 0.5).float()  # Threshold predictions at 0.5

    return loss, preds, labels

def train(model, train_loader, criterion, optimizer, epochs):
    model.train() # Set model to training mode

    for epoch in range(epochs): # Specify the number of iterations to train for
        all_preds, all_labels = [], []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            optimizer.zero_grad()
            loss, preds, labels = process_forward_pass(model, batch, criterion)

            loss.backward()
            optimizer.step()
            # (backward phase, upd. weights)

            all_preds.extend(preds.numpy())   # Store predictions and labels for evaluation
            all_labels.extend(labels.numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds)

        print(f"Acc={accuracy:.2f}%, F1={f1:.4f}")

After your model trains, you want to see how well it performs on **unseen data.** Meaning, if this were a live hospital NEEDING your predictions to classify patients with pneumonia, how well would it do?


You simply have to run this cell; all the code is implemented for you (Assuming `process_forward_phase` works fine). 😊

In [23]:
def test(model, test_loader, criterion):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad(): # Disable gradient computation
        for batch in test_loader:
            loss, preds, labels = process_forward_pass(model, batch, criterion)
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Final Test Results: Acc={accuracy:.2f}%, F1={f1:.4f}")

#### 📌***TASK 4 - TRAIN MODEL***

We're close!!! We simply need to instantiate the `model`, define a suitable `criterion` (loss), and use an `optimizer` (thing to speed up backpropagation).

In [24]:
import torch.optim as optim

criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) # Weight decay is optional. But is it? 🤔

train(model, train_loader, criterion, optimizer, epochs=5)

Epoch 1/5: 100%|██████████| 326/326 [07:05<00:00,  1.30s/it]


Acc=0.85%, F1=0.9037


Epoch 2/5: 100%|██████████| 326/326 [06:54<00:00,  1.27s/it]


Acc=0.92%, F1=0.9471


Epoch 3/5: 100%|██████████| 326/326 [06:40<00:00,  1.23s/it]


Acc=0.94%, F1=0.9619


Epoch 4/5: 100%|██████████| 326/326 [07:24<00:00,  1.36s/it]


Acc=0.95%, F1=0.9678


Epoch 5/5: 100%|██████████| 326/326 [08:31<00:00,  1.57s/it]

Acc=0.96%, F1=0.9719


Last step: evaluate your model's performance. Remember, you get **1,000,000** brownie points 🍫 if you beat Adam's **`f1_score:`0.8549**.

In [25]:
test(model, test_loader, criterion)

Final Test Results: Acc=0.75%, F1=0.8323
